In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 5, 15), datetime.date(2022, 5, 14))

In [2]:
yesterday = today - timedelta(days=2)
yesterday

datetime.date(2022, 5, 13)

In [3]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [4]:
name = 'IVL'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [5]:
sqlIns = """
INSERT INTO dividend
VALUES('IVL',0.25,0.25,0.25,0.25,1.00,0,0,00,'2022-05-27','2022-06-10','',0)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [5]:
def update_dividend1(q1,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q1 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q1,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [6]:
name = 'JASIF'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JASIF,0.2500,0.2200,0.2400,0.2300,0.9400,"100,000",2022-05-24,2022-06-08,2,1


In [8]:
q1 = 0.125
XDATE = '2022-05-20'
PAIDDATE = '2022-06-08
'
actual = 1

update_dividend1(q1,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [7]:
def update_dividend2(shares,q1,actual,name):
    sql = "UPDATE dividend SET shares = %s, q1 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q1, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [8]:
name = 'STA'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,STA,0.6500,1.2500,1.2500,1.0000,4.1500,"20,000",2022-04-19,2022-05-06,,1


In [9]:
shares = 20000
q1 = 0
actual = 0

In [11]:
update_dividend2(shares,q1,actual,name)

'Records updated = 1'

### Toggle actual status

In [4]:
name = 'DCC'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [10]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("MC")
'''
rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [5]:
sql = """
SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '2022-05-13'
ORDER BY name



In [6]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [7]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,DCC,0.0550,"60,000","3,300.00","2,970.00",2022-05-13,2022-05-31,"177,600.00",1.67%,0
1,DIF,0.2600,"50,000","13,000.00","11,700.00",2022-05-17,2022-06-06,"735,000.00",1.59%,1
2,HREIT,0.1738,"40,100","6,969.38","6,272.44",2022-03-04,2022-03-30,"358,895.00",1.75%,0
3,JASIF,0.2300,"100,000","23,000.00","20,700.00",2022-05-24,2022-06-08,"1,000,000.00",2.07%,1
4,SENA,0.1300,"105,000","13,650.00","12,285.00",2022-05-11,2022-05-27,"470,400.00",2.61%,0
5,WHART,0.1915,"20,000","3,830.00","3,447.00",2022-06-01,2022-06-15,"246,000.00",1.40%,1


In [8]:
dividend.net.sum()

57374.44

In [8]:
file_name = 'dividend-q1.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(box_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(one_file, index=False)

### End of output to cloud